In [3]:
import torch
import torch.nn as nn
import time
import math

# 之前定义的导入歌词数据函数，直接拿来使用
def load_data_jay_lyrics():
    with open('./dataset/jaychou_lyrics.txt', encoding='utf-8') as f:
        corpus_chars = f.read()
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_jay_lyrics()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 指定CPU模式

In [4]:
# 初始化模型参数
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
# num_inputs: d
# num_hiddens: h, 隐藏单元的个数是超参数
# num_outputs: q

# def get_params():
#     def _one(shape):
#         param = torch.zeros(shape, device=device, dtype=torch.float32)
#         nn.init.normal_(param, 0, 0.01)
#         return torch.nn.Parameter(param)

#     # 隐藏层参数
#     W_xh = _one((num_inputs, num_hiddens))
#     W_hh = _one((num_hiddens, num_hiddens))
#     b_h = torch.nn.Parameter(torch.zeros(num_hiddens, device=device))
#     # 输出层参数
#     W_hq = _one((num_hiddens, num_outputs))
#     b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device))
#     return (W_xh, W_hh, b_h, W_hq, b_q)

In [6]:
# 查看简单效果
rnn_layer = nn.RNN(input_size=vocab_size, hidden_size=num_hiddens)
num_steps, batch_size = 35, 2
X = torch.rand(num_steps, batch_size, vocab_size)  # 输入数据
state = None
Y, state_new = rnn_layer(X, state)
print(Y.shape, state_new.shape)

torch.Size([35, 2, 256]) torch.Size([1, 2, 256])


In [7]:
def one_hot(x, n_class, dtype=torch.float32):
    result = torch.zeros(x.shape[0], n_class, dtype=dtype, device=x.device)  # shape: (n, n_class)
    result.scatter_(1, x.long().view(-1, 1), 1)  # result[i, x[i, 0]] = 1
    return result
def to_onehot(X, n_class):
    return [one_hot(X[:, i], n_class) for i in range(X.shape[1])]
    

# 接下来定义完整的RNN语言模型
class RNNModel(nn.Module):
    def __init__(self, rnn_layer, vocab_size):
        super(RNNModel, self).__init__()
        self.rnn = rnn_layer
        # 判断是否双向RNN，此处非双向
        self.hidden_size = rnn_layer.hidden_size * (2 if rnn_layer.bidirectional else 1)
        self.vocab_size = vocab_size
        self.dense = nn.Linear(self.hidden_size, vocab_size)
    
    def forward(self, inputs, state):  # 前向传播计算
        x = to_onehot(inputs, vocab_size)
        # stack作用在于将num_steps个(batch_size, vocab_size)的tensor转化为完整的tensor
        x = torch.stack(x)  # x的shape： (num_steps, batch_size, vocab_size)
        hiddens, state = self.rnn(x, state)  # hiddens为输出  state为最新的隐层状态
        # 由于以上的输入x为3D的tensor，因此需要转化为2D的tensor
        hiddens = hiddens.view(-1, hiddens.shape[-1])  # hiddens.shape: (num_steps * batch_size, hidden_size)
        outputs = self.dense(hiddens)
        return outputs, state

In [8]:
# 预测函数
def predict_rnn_pytorch(prefix, num_chars, model, vocab_size, device, idx_to_char,
                      char_to_idx):
    state = None
    output = [char_to_idx[prefix[0]]]  # output记录prefix加上预测的num_chars个字符
    for t in range(num_chars + len(prefix) - 1):
        X = torch.tensor([output[-1]], device=device).view(1, 1)
        (Y, state) = model(X, state)  # 前向计算不需要传入模型参数
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(Y.argmax(dim=1).item())
    return ''.join([idx_to_char[i] for i in output])


In [20]:
# nn.RNN 已内置权重和偏置参数，自动随机初始化
# 观察简单效果
model = RNNModel(rnn_layer, vocab_size) 
predict_rnn_pytorch('分开', 10, model, vocab_size, device, idx_to_char, char_to_idx)

'分开瑰鼻光光瑰美光光外鼻'

In [15]:
# 前面定义的相邻采样函数
def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_len = len(corpus_indices) // batch_size * batch_size  # 保留下来的序列的长度
    corpus_indices = corpus_indices[: corpus_len]  # 仅保留前corpus_len个字符
    indices = torch.tensor(corpus_indices, device=device)
    indices = indices.view(batch_size, -1)  # resize成(batch_size, )
    batch_num = (indices.shape[1] - 1) // num_steps
    for i in range(batch_num):
        i = i * num_steps
        X = indices[:, i: i + num_steps]
        Y = indices[:, i + 1: i + num_steps + 1]
        yield X, Y

# 前面定义的梯度裁剪函数
def grad_clipping(params, theta, device):
    norm = torch.tensor([0.0], device=device)
    for param in params:  
        norm += (param.grad.data ** 2).sum() # 梯度值总和
    norm = norm.sqrt().item()
    if norm > theta:  
        for param in params:
            param.grad.data *= (theta / norm)

        
# 训练模型函数
def train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes):
    loss = nn.CrossEntropyLoss()  # 损失函数
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)  # 优化器
    model.to(device)
    for epoch in range(num_epochs):
        state = None  # 初始化隐藏层
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_consecutive(corpus_indices, batch_size, num_steps, device) # 相邻采样
        for X, Y in data_iter:        
            if state is not None:
                # 使用detach函数从计算图分离隐藏状态
                if isinstance (state, tuple):  # 判断state是否tuple类型
                    state[0].detach_()
                    state[1].detach_()
                else:
                    state.detach_()
            (output, state) = model(X, state) # output.shape: (num_steps * batch_size, vocab_size)
            y = torch.flatten(Y.T)  # Y转置后按行展开
            l = loss(output, y.long())
            optimizer.zero_grad()  # 梯度清零
            l.backward()  # 梯度传播
            grad_clipping(model.parameters(), clipping_theta, device)  # 裁剪梯度，避免梯度爆炸
            optimizer.step()
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn_pytorch(
                    prefix, pred_len, model, vocab_size, device, idx_to_char,
                    char_to_idx))

In [16]:
# 观察效果
num_epochs, batch_size, lr, clipping_theta = 250, 32, 1e-3, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                            corpus_indices, idx_to_char, char_to_idx,
                            num_epochs, num_steps, lr, clipping_theta,
                            batch_size, pred_period, pred_len, prefixes)

KeyboardInterrupt: 